### Modelo predictivo de series temporales con Azure (Time series forecasting): Esperanza de Vida 2024

El modelo entrenado en Azure ML atiende a una predicción de series temporales multivariadas de la esperanza de vida en las capitales europeas. Esto significa que se consideran múltiples series temporales (una para cada ciudad), y se está pronosticando la variable de esperanza de vida para cada serie.

In [1]:

import joblib
import pickle
import azureml

model = joblib.load('model_series.pkl')

In [2]:
model

ForecastingPipelineWrapper(pipeline=Pipeline(steps=[('timeseriestransformer',
                                                     TimeSeriesTransformer(country_or_region=None, drop_column_names=[], featurization_config=FeaturizationConfig(blocked_transformers=None, column_purposes=None, dataset_language=None, prediction_transform_type=None, transformer_params=None), force_time_index_features=None, freq='AS-J...
                                                     Average(timeseries_param_dict={'time_column_name': 'Año', 'grain_column_names': ['Ciudad'], 'target_column_name': 'Esperanza vida', 'drop_column_names': [], 'overwrite_columns': True, 'dropna': False, 'transform_dictionary': {'min': '_automl_target_col', 'max': '_automl_target_col', 'mean': '_automl_target_col'}, 'max_horizon': 1, 'origin_time_colname': 'origin', 'country_or_region': None, 'n_cross_validations': 5, 'short_series_handling': True, 'max_cores_per_iteration': -1, 'feature_lags': None, 'target_aggregation_function': 'mean', 'cv_step_size': 1, 'window_size': 'auto', 'lags': {'_automl_target_col': [8]}, 'iteration_timeout_minutes': 30, 'seasonality': 1, 'use_stl': None, 'freq': 'AS-JAN', 'short_series_handling_configuration': 'auto', 'target_lags': [8], 'target_rolling_window_size': 0, 'arimax_raw_columns': ['Año', 'Ciudad'], 'lagging_columns': ['_automl_target_col_lag8AS-JAN']}))]),
                           stddev=[0.0036502869881320545])

In [4]:
# Importamos el dataset original

import pandas as pd


df = pd.read_csv('inputs/df_life_exp.csv')

In [5]:
df

,Ciudad,Año,Esperanza vida
0,Bruselas,2014,81.40
1,Sofía,2014,74.50
2,Copenhague,2014,80.70
3,Berlín,2014,81.20
4,Tallin,2014,77.40
...,...,...,...
265,Liubliana,2023,81.85
266,Bratislava,2023,78.00
267,Helsinki,2023,82.48
268,Estocolmo,2023,83.33


In [7]:
# Creamos una nueva serie temporal para añadir dataframe. Esta serie temporal corresponderá al año 2024 de cada ciudad y, como en total hay 27, creamos una lista con 27 veces el año 2024.

lista_2024 = ['2024-01-01']*27

lista_2024

['2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01',
 '2024-01-01']

In [9]:
# Creamos una lista con las ciudades que añadiremos al df original
ciudades = df['Ciudad'].unique().tolist()
ciudades

['Bruselas',
 'Sofía',
 'Copenhague',
 'Berlín',
 'Tallin',
 'Dublín',
 'Atenas',
 'Madrid',
 'París',
 'Zagreb',
 'Roma',
 'Nicosia',
 'Riga',
 'Vilna',
 'Luxemburgo',
 'Budapest',
 'Valeta',
 'Ámsterdam',
 'Viena',
 'Varsovia',
 'Lisboa',
 'Bucarest',
 'Liubliana',
 'Bratislava',
 'Helsinki',
 'Estocolmo',
 'Praga']

In [10]:
# Creamos un df con las ciudades y el año 2024

df2 = pd.DataFrame(data=list(zip(ciudades, lista_2024)), columns=['Ciudad', 'Año'], dtype='datetime64[ns]')


In [11]:
# hacemos la predicción para cada año y ciudad

model.forecast(df2, ignore_data_errors=True)[0]

array([81.60709596, 74.37919132, 81.17975155, 81.07816633, 78.2776158 ,
       82.22121733, 81.4199137 , 83.35918687, 82.66261101, 77.9622717 ,
       83.14005143, 82.11075432, 74.90285752, 75.42111092, 82.51814824,
       76.01091514, 82.50580082, 81.82767037, 81.59473663, 77.54759948,
       81.58505362, 75.00023412, 81.20470146, 77.03009609, 81.75830481,
       82.71319935, 78.70287576])

In [12]:
# Convertimos la lista anterior en un dataframe

df_forecast = pd.DataFrame(model.forecast(df2, ignore_data_errors=True)[0], columns=['Esperanza vida'])

df_forecast

c:\Users\Adriana Sanjuán\Desktop\PROYECTO_FINAL\entorno_cities\lib\site-packages\azureml\training\tabular\models\forecasting_pipeline_wrapper_base.py:1409: UserWarning: Missing prediction context. There is a gap between the end of training period and the beginning of the provided test set; the testing period is expected to directly follow the training period since the forecaster may depend on past values. If this gap is expected, please run forecast() with the `ignore_data_errors=True` option. In this case, the target and feature values in the gap will be imputed.
  warnings.warn(error.error_message)


,Esperanza vida
0,81.61
1,74.38
2,81.18
3,81.08
4,78.28
5,82.22
6,81.42
7,83.36
8,82.66
9,77.96


In [13]:
# Añadimos las predicciones del año 2024 a la lista de ciudades y años

df_forecast = pd.concat([df2, df_forecast], axis=1)

df_forecast

,Ciudad,Año,Esperanza vida
0,Bruselas,2024-01-01,81.61
1,Sofía,2024-01-01,74.38
2,Copenhague,2024-01-01,81.18
3,Berlín,2024-01-01,81.08
4,Tallin,2024-01-01,78.28
5,Dublín,2024-01-01,82.22
6,Atenas,2024-01-01,81.42
7,Madrid,2024-01-01,83.36
8,París,2024-01-01,82.66
9,Zagreb,2024-01-01,77.96


In [14]:
df_forecast['Año'] = df_forecast['Año'].dt.year # Convertimos la columna Año a formato año

In [15]:
df_forecast

,Ciudad,Año,Esperanza vida
0,Bruselas,2024,81.61
1,Sofía,2024,74.38
2,Copenhague,2024,81.18
3,Berlín,2024,81.08
4,Tallin,2024,78.28
5,Dublín,2024,82.22
6,Atenas,2024,81.42
7,Madrid,2024,83.36
8,París,2024,82.66
9,Zagreb,2024,77.96


In [16]:
# Unimos hacia abajo el df original con el df_forecast

df = pd.concat([df, df_forecast], axis=0)

df

,Ciudad,Año,Esperanza vida
0,Bruselas,2014,81.40
1,Sofía,2014,74.50
2,Copenhague,2014,80.70
3,Berlín,2014,81.20
4,Tallin,2014,77.40
...,...,...,...
22,Liubliana,2024,81.20
23,Bratislava,2024,77.03
24,Helsinki,2024,81.76
25,Estocolmo,2024,82.71


In [17]:
# Guardamos el df en un csv

df.to_csv('df_life_exp_2024.csv', index=False)